# Домашняя работа №5. Логистическая регрессия

Цель работы: Обучить модель логистической регрессии с настройкой гиперпараметров.

### Работа с набором данных для задачи классификации (целевая переменная - класс объекта)

## Задание №1

Цель: Получить чистые данные

#### Предварительная подготовка данных
* Заполнить пропуски в данных подходящими значениями (например, 0 для числовых значений, other для категориальных значений).
* Построить график количества значений по каждому классу.
* Построить графики распределения данных.
* Проверить датасет на дубликаты. Удалить при наличии.
* Закодировать категориальные значения. Например, создать новые булевые признаки (OneHotEncoder), или назначить число для каждого значения (LabelEncoder).
* Удалить столбцы с уникальными значениями, при наличии.
* К числовым признакам применить масштабирование со средним нулевым значением.
* Разделить на тренировочную и обучающую выборки.

## Задание №2

#### Обучение модели логистической регресии
* Обучите модель логистической регресии в Sklearn с параметрами по умолчанию.
* Вычислить метрики recall, precision, f1 на тренировочной и тестовой выборках.
* Сделать вывод о качестве обученной модели.

## Задание №3

#### Настройка модели
* Обучите модель логистической регресии с подходящей для датасета и результата обучения типом регуляризации(l1, l2, elasticnet).
* Вычислить метрики полученной модели (f1, recall, precision).
* Сделать вывод о качестве обученной модели.
* Приведите график ROC-кривой.

## Задание №4

#### Настройка гиперпараметров модели.
*  Обучите модель логистической регресии с разным количеством итераций, параметром регуляризации и величной штрафа(параметр С).
* Вычислить метрики Recall, Precision, F1 на тренировочной и тестовой выборках.
* Сделать вывод о качестве обученной модели.

# Решение домашней работы №5. Логистическая регрессия

In [58]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score

In [59]:
df = pd.read_csv('/content/titanic/train.csv')

## Задание №1

####Подготовим данные
Начнем с пропусков

In [60]:
median_age = df['Age'].median()
df['Age'] = df['Age'].fillna(median_age)

most_common_embarked = df['Embarked'].value_counts().index[0]
df['Embarked'] = df['Embarked'].fillna(most_common_embarked)

Поработаем с категориальными признаками

In [61]:
label_encoder = LabelEncoder()
df['Sex'] = label_encoder.fit_transform(df['Sex'])
df['Embarked'] = label_encoder.fit_transform(df['Embarked'])

Масштабируем с помощью StandartScaler числовые признаки

In [62]:
scaler = StandardScaler()
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare']
scaled_data = scaler.fit_transform(df[numerical_features])
df[numerical_features] = scaled_data

Удалим столбцы уникальных значений

In [63]:
df = df.drop(columns=['Ticket', 'Name', 'Cabin'])

In [64]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,-0.565736,0.432793,-0.473674,-0.502445,2
1,2,1,1,0,0.663861,0.432793,-0.473674,0.786845,0
2,3,1,3,0,-0.258337,-0.474545,-0.473674,-0.488854,2
3,4,1,1,0,0.433312,0.432793,-0.473674,0.420730,2
4,5,0,3,1,0.433312,-0.474545,-0.473674,-0.486337,2


Разделим на test и train

In [66]:
X = df.drop(['PassengerId', 'Survived'], axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,-0.565736,0.432793,-0.473674,-0.502445,2
1,1,0,0.663861,0.432793,-0.473674,0.786845,0
2,3,0,-0.258337,-0.474545,-0.473674,-0.488854,2
3,1,0,0.433312,0.432793,-0.473674,0.420730,2
4,3,1,0.433312,-0.474545,-0.473674,-0.486337,2


In [68]:
y.head()

,Survived
0,0
1,1
2,1
3,1
4,0


## Задание №2

####Обучение модели

In [69]:
logreg = LogisticRegression(random_state=42)
logreg.fit(X_train, y_train)

LogisticRegression(random_state=42)

####Прогноз на тестовом множестве

In [70]:
y_pred = logreg.predict(X_test)

####Оценка качества

In [71]:
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Precision: {precision:.2f}')
print(f'F1 Score: {f1:.2f}')

Accuracy: 0.81
Recall: 0.74
Precision: 0.79
F1 Score: 0.76


####Вывод
У модели хорошие метрики обучения

## Задание №3

####L1

In [72]:
logreg_l1 = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42)
logreg_l1.fit(X_train, y_train)
y_pred_l1 = logreg_l1.predict(X_test)

#####Оценка качества

In [38]:
accuracy_l1 = accuracy_score(y_test, y_pred_l1)
recall_l1 = recall_score(y_test, y_pred_l1)
precision_l1 = precision_score(y_test, y_pred_l1)
f1_l1 = f1_score(y_test, y_pred_l1)

print(f'Accuracy: {accuracy_l1:.2f}')
print(f'Recall: {recall_l1:.2f}')
print(f'Precision: {precision_l1:.2f}')
print(f'F1 Score: {f1_l1:.2f}')

Accuracy: 0.80
Recall: 0.72
Precision: 0.78
F1 Score: 0.75


####L2

In [39]:
logreg_l2 = LogisticRegression(penalty='l2', C=1.0, random_state=42)
logreg_l2.fit(X_train, y_train)
y_pred_l2 = logreg_l2.predict(X_test)

#####Оценка качества

In [40]:
accuracy_l2 = accuracy_score(y_test, y_pred_l2)
recall_l2 = recall_score(y_test, y_pred_l2)
precision_l2 = precision_score(y_test, y_pred_l2)
f1_l2 = f1_score(y_test, y_pred_l2)

print(f'Accuracy: {accuracy_l2:.2f}')
print(f'Recall: {recall_l2:.2f}')
print(f'Precision: {precision_l2:.2f}')
print(f'F1 Score: {f1_l2:.2f}')

Accuracy: 0.81
Recall: 0.74
Precision: 0.79
F1 Score: 0.76


####ElasticNet

In [42]:
logreg_elastic = LogisticRegression(penalty='elasticnet', l1_ratio=0.5, C=1.0, solver='saga', random_state=42)
logreg_elastic.fit(X_train, y_train)
y_pred_elastic = logreg_elastic.predict(X_test)

#####Оценка качества

In [43]:
accuracy_elastic = accuracy_score(y_test, y_pred_elastic)
recall_elastic = recall_score(y_test, y_pred_elastic)
precision_elastic = precision_score(y_test, y_pred_elastic)
f1_elastic = f1_score(y_test, y_pred_elastic)

print(f'Accuracy: {accuracy_elastic:.2f}')
print(f'Recall: {recall_elastic:.2f}')
print(f'Precision: {precision_elastic:.2f}')
print(f'F1 Score: {f1_elastic:.2f}')

Accuracy: 0.81
Recall: 0.74
Precision: 0.79
F1 Score: 0.76


####Графики

## Задание №4

In [48]:
params = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1.0, 10.0],
    'max_iter': [100, 200, 500],
    'solver': ['liblinear', 'saga', 'lbfgs']
}

In [49]:
params = [
    {'penalty': ['l1'], 'C': [0.01, 0.1, 1.0, 10.0], 'solver': ['liblinear', 'saga'], 'max_iter': [100, 200, 500]},
    {'penalty': ['l2'], 'C': [0.01, 0.1, 1.0, 10.0], 'solver': ['lbfgs'], 'max_iter': [100, 200, 500]}
]

####Лучшие параметры

In [50]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), params, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_

####Переобучение модели

In [51]:
best_logreg = LogisticRegression(**best_params)
best_logreg.fit(X_train, y_train)
y_pred_best = best_logreg.predict(X_test)

####Оценка качества

In [52]:
accuracy_best = accuracy_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)

print(f'Accuracy: {accuracy_best:.2f}')
print(f'Recall: {recall_best:.2f}')
print(f'Precision: {precision_best:.2f}')
print(f'F1 Score: {f1_best:.2f}')

Accuracy: 0.80
Recall: 0.72
Precision: 0.78
F1 Score: 0.75


####Вывод
У модели хорошие метрики обучения